## [LangChain Quickstart Guide](https://python.langchain.com/docs/get_started/quickstart)

Load the environmental variables form .env file

In [1]:
import json
import textwrap
from dotenv import load_dotenv
import os
load_dotenv()

True

We need to first: `pip install langchain`

In [2]:
def printlongtext(text, max_lines=10):
    short_strings = textwrap.wrap(text, 80)
    lines = len(short_strings)
    if lines < max_lines:
        max_lines = lines
    for index in range(0, max_lines):
        print(short_strings[index])
        

### LLM Chain
We first need to: `pip install langchain-openai`<br/>
Make sure we have the openai credentials in the .env file.

In [52]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    # model="gpt-4",
    model="gpt-3.5-turbo",
    temperature=0,
)

In [55]:
from langchain_community.llms import Ollama
llm = Ollama(
    model = "llama2"
)

In [57]:
result =llm.invoke("what's the difference between langchain and llamaindex?")

Examine the result

In [60]:
# print(result.content)
print(result)
# print the json object as a string with pretty formatting with indent of 4, also make it wrap long lines
# print(json.dumps(result.json(), indent=4))


Langchain and Llamaindex are two different tools that serve distinct purposes in the context of natural language processing (NLP) and machine learning (ML). Here's a comparison of the two:

1. Language Modeling: Langchain is primarily designed for language modeling tasks, such as text generation, language translation, and language understanding. On the other hand, Llamaindex is focused on index-based language modeling, which involves creating an index of pre-defined phrases or sentences to improve the efficiency of language processing tasks.
2. Architecture: Langchain utilizes a transformer-based architecture, similar to BERT and RoBERTa, while Llamaindex leverages an index-based architecture that stores pre-defined phrase embeddings in a compact format.
3. Training: Langchain requires extensive training on large datasets to learn the patterns and relationships between words in a language, while Llamaindex can be trained quickly and easily using a small dataset of pre-defined phrases.

In [6]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

Combine prompt and llm into a chain.

In [7]:
chain = prompt | llm 

In [8]:
chain.invoke({"input": "how can langsmith help with testing?"})

AIMessage(content='Langsmith can help with testing in several ways:\n\n1. Automation: Langsmith provides powerful automation tools and frameworks that can help automate testing processes, such as test case execution, regression testing, and performance testing. This can save time and effort in the testing process and improve efficiency.\n\n2. Test data generation: Langsmith can generate realistic and diverse test data that can be used to validate the functionality and performance of the software under test. This can help in identifying potential issues and improving test coverage.\n\n3. Test case management: Langsmith offers test case management capabilities that allow testers to organize, prioritize, and track test cases effectively. This can streamline the testing process and ensure that all necessary tests are executed.\n\n4. Reporting and analytics: Langsmith provides comprehensive reporting and analytics features that can help testers analyze test results, identify trends, and mak

In [9]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [10]:
chain = prompt | llm | output_parser

In [11]:
result = chain.invoke({"input": "how can langsmith help with testing?"})

In [14]:
# result is no longer an json object, it is a string
print(result)

Langsmith is a powerful tool that can greatly assist with testing in a variety of ways. Here are some ways in which Langsmith can help with testing:

1. **Automated Testing:** Langsmith can be used to automate testing tasks, such as running tests, comparing output, and generating reports. This can help streamline the testing process and ensure more consistent and thorough testing coverage.

2. **Test Data Generation:** Langsmith can generate a wide variety of test data, including strings, numbers, dates, and more. This can be especially useful for creating test cases with different input values to ensure comprehensive testing coverage.

3. **Mocking and Stubbing:** Langsmith can be used to create mock objects or stubs for dependencies in your tests. This can help isolate the code under test and make testing easier by simulating the behavior of external components.

4. **Performance Testing:** Langsmith can be used to simulate load and stress testing scenarios by generating a large volu

### Retrival Chain

WebBaseLoader uses beautifulsoup to scrape text from a web page.<br/>
We first need to: `pip install beautifulsoup4`

In [15]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com")

docs = loader.load()

In [16]:
printlongtext(docs[0].page_content.strip("\n"), 20)
print("-" * 80)
print(json.dumps(docs[0].metadata, indent=4))

LangSmith | 🦜️🛠️ LangSmith      Skip to main content🦜️🛠️ LangSmith DocsLangChain
Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser
GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt
HubLangSmithOn this pageLangSmithIntroduction​LangSmith is a platform for
building production-grade LLM applications.It lets you debug, test, evaluate,
and monitor chains and intelligent agents built on any LLM framework and
seamlessly integrates with LangChain, the go-to open source framework for
building with LLMs.LangSmith is developed by LangChain, the company behind the
open source LangChain framework.Quick Start​Tracing: Get started with the
tracing quick start.Evaluation: Get started with the evaluation quick start.Next
Steps​Check out the following sections to learn more about LangSmith:User Guide:
Learn about the workflows LangSmith supports at each stage of the LLM
application lifecycle.Setup: Learn how to create an account, obtain an API key,


This time uses a local embedding model from ollama.<br/>
First we need to install ollama.<br/>
Then we need to pull the model we need to use: <br/>
`ollama pull <model_name>` <br/>
`ollama run llama2`

In [20]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings()

Need to do this first: `pip install faiss-cpu`

In [22]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [23]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [24]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [25]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...

LangSmith can help with testing by allowing users to debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework. It provides tracing and evaluation capabilities to assist with testing the applications.


In [76]:
printlongtext(response["answer"])

LangSmith can help with testing by allowing users to debug, test, evaluate, and
monitor chains and intelligent agents built on any LLM framework. It provides
tracing and evaluation capabilities, as well as integration with the LangChain
framework for building with LLMs. Users can also access the Prompt Hub, a prompt
management tool built into LangSmith, for testing purposes.


## Conversation Retrieval Chain

In [26]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [27]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
result = retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

In [28]:
printlongtext(result[0].page_content)

LangSmith | 🦜️🛠️ LangSmith      Skip to main content🦜️🛠️ LangSmith DocsLangChain
Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser
GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt
HubLangSmithOn this pageLangSmithIntroduction​LangSmith is a platform for
building production-grade LLM applications.It lets you debug, test, evaluate,
and monitor chains and intelligent agents built on any LLM framework and
seamlessly integrates with LangChain, the go-to open source framework for
building with LLMs.LangSmith is developed by LangChain, the company behind the
open source LangChain framework.Quick Start​Tracing: Get started with the
tracing quick start.Evaluation: Get started with the evaluation quick start.Next


In [29]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [30]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
result = retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

In [31]:
printlongtext(result["answer"])

LangSmith can help you test your LLM applications by providing tracing and
evaluation capabilities. With tracing, you can debug and monitor your chains and
intelligent agents, while evaluation allows you to test and assess the
performance of your applications. LangSmith seamlessly integrates with
LangChain, making it easier to build, test, and monitor your LLM applications
effectively.


## Agent

In [32]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

Need Tavily API Key for this. (Check .env file)

In [33]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

In [34]:
tools = [retriever_tool, search]

First need to: `pip install langchainhub`

In [35]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [36]:
result = agent_executor.invoke({"input": "how can langsmith help with testing?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'how can LangSmith help with testing'}`


LangSmith | 🦜️🛠️ LangSmith





Skip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubLangSmithOn this pageLangSmithIntroduction​LangSmith is a platform for building production-grade LLM applications.It lets you debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework and seamlessly integrates with LangChain, the go-to open source framework for building with LLMs.LangSmith is developed by LangChain, the company behind the open source LangChain framework.Quick Start​Tracing: Get started with the tracing quick start.Evaluation: Get started with the evaluation quick start.Next Steps​Check out the following sections to learn more about LangSmith:User Guide: Learn about the workfl

In [37]:
# print the json object as a string with pretty formatting with indent of 4, also make it wrap long lines
print(json.dumps(result, indent=4))
print("-" * 80)
printlongtext(result["output"], 20)

{
    "input": "how can langsmith help with testing?",
    "output": "LangSmith is a platform for building production-grade LLM (Large Language Model) applications. It can help with testing by providing capabilities for debugging, testing, evaluating, and monitoring chains and intelligent agents built on any LLM framework. LangSmith seamlessly integrates with LangChain, an open-source framework for building with LLMs. \n\nSome ways LangSmith can help with testing include:\n- Tracing: LangSmith offers tracing capabilities to help you trace and debug your LLM applications.\n- Evaluation: LangSmith provides evaluation capabilities to assess the performance and effectiveness of your LLM models during testing.\n\nFor more detailed information on how LangSmith can assist with testing, you can refer to the LangSmith documentation and user guide."
}
--------------------------------------------------------------------------------
LangSmith is a platform for building production-grade LLM (Large 

In [38]:
sf_weather = agent_executor.invoke({"input": "what is the weather in SF?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`


[{'url': 'https://forecast.weather.gov/zipcity.php?inputstring=San+Francisco,CA', 'content': 'NOAA National Weather Service National Weather Service. Toggle navigation. HOME; FORECAST . Local; Graphical; Aviation; Marine; Rivers and Lakes; Hurricanes; Severe Weather; Fire Weather; ... San Francisco CA 37.77°N 122.41°W (Elev. 131 ft) Last Update: 12:26 am PST Mar 2, 2024. Forecast Valid: 1am PST Mar 2, 2024-6pm PST Mar 8, 2024 .'}, {'url': 'https://weather.com/weather/tenday/l/San Francisco CA USCA0987:1:US', 'content': "Comfy & Cozy\nThat's Not What Was Expected\nOutside\n'No-Name Storms' In Florida\nGifts From On High\nWhat To Do For Wheezing\nSurviving The Season\nStay Safe\nAir Quality Index\nAir quality is considered satisfactory, and air pollution poses little or no risk.\n Health & Activities\nSeasonal Allergies and Pollen Count Forecast\nNo pollen detect

In [39]:
printlongtext(sf_weather["output"], 20)

You can check the current weather in San Francisco by visiting the [NOAA
National Weather Service
website](https://forecast.weather.gov/zipcity.php?inputstring=San+Francisco,CA)
or [The Weather Channel website](https://weather.com/weather/tenday/l/San
Francisco CA USCA0987:1:US).


In [40]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
result_ls = agent_executor.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'LangSmith LLM application testing'}`


LangSmith | 🦜️🛠️ LangSmith





Skip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubLangSmithOn this pageLangSmithIntroduction​LangSmith is a platform for building production-grade LLM applications.It lets you debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework and seamlessly integrates with LangChain, the go-to open source framework for building with LLMs.LangSmith is developed by LangChain, the company behind the open source LangChain framework.Quick Start​Tracing: Get started with the tracing quick start.Evaluation: Get started with the evaluation quick start.Next Steps​Check out the following sections to learn more about LangSmith:User Guide: Learn about the workflow

In [41]:
printlongtext(result_ls["output"], 20)

LangSmith is a platform for building production-grade LLM applications. It
allows you to debug, test, evaluate, and monitor chains and intelligent agents
built on any LLM framework. LangSmith seamlessly integrates with LangChain, an
open-source framework for building with LLMs. You can learn more about LangSmith
by checking out their user guide, setup instructions, pricing, self-hosting
options, tracing capabilities, evaluation capabilities, and the Prompt Hub. If
you are interested in testing your LLM applications, LangSmith can help you with
that process.


## Serving with LangServe

Refer to serv.py

Also needs to: `pip install "langserve[all]"`

# LangServe client

In [43]:
from langserve import RemoteRunnable

remote_chain = RemoteRunnable("http://localhost:8080/agent/")
result =remote_chain.invoke({
    "input": "how can langsmith help with testing?",
    "chat_history": []  # Providing an empty list as this is the first call
})

In [44]:
print(result["output"])

LangSmith is a platform for building production-grade LLM (Large Language Model) applications. It can help with testing by providing capabilities for debugging, testing, evaluating, and monitoring chains and intelligent agents built on any LLM framework. LangSmith seamlessly integrates with LangChain, an open-source framework for building with LLMs. 

Some ways LangSmith can help with testing include:
- Tracing: LangSmith offers tracing capabilities to help you trace and debug your LLM applications.
- Evaluation: LangSmith provides evaluation capabilities to assess the performance and effectiveness of your LLM models during testing.

You can learn more about how LangSmith can assist with testing by exploring its user guide, setup instructions, pricing, and additional resources available on the LangSmith platform.
